## Practical 2 - task 2

In [ ]:
import datetime
import hashlib
import json

In [ ]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.create_block(self.get_temp_block('0'))

    def create_block(self, block):
        self.chain.append(block)
        self.transactions = []
        return block

    def get_previous_block(self):
        return self.chain[-1]

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hash(encoded_block)

    def proof_of_work(self, temp_block):
        new_proof = 1
        check_proof = False
        while check_proof is False:
            temp_block['nonce'] = new_proof
            hash_operation = self.hash(temp_block)
            if hash_operation.startswith('000'):
                check_proof = True
            else:
                new_proof += 1
        return temp_block

    def create_transaction(self, sender, receiver, amount):
        self.transactions.append({'sender': sender,
                                  'receiver': receiver,
                                  'amount': amount})
        previous_block = self.get_previous_block() # needs to be changed
        return "Transaction will be added to Block " + str(previous_block['index'] + 1)

    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                print(block['previous_hash'])
                print(self.hash(previous_block))
                return False
            if not self.hash(block).startswith('000'):
                print(self.hash(block))
                return False
            previous_block = block
            block_index += 1
        return True

    def get_temp_block(self, previous_hash):
        block = {'index': len(self.chain) + 1,
                 'nonce': 1,
                 'timestamp': str(datetime.datetime.now()),
                 'previous_hash': previous_hash,
                 'transactions': self.transactions,
                 'merkle_root': get_merkle_root(self.transactions)}

        return block

In [ ]:
# Create merkle tree
def get_merkle_root(transactions):
    if len(transactions) == 0:
        return None

    if len(transactions) == 1:
        return hash(transactions[0])

    # hash all transactions before building merkle tree
    hashed_transactions = []
    for i in range(len(transactions)):
        hashed_transactions.append(hash(transactions[i]))

    # build merkle tree
    level = 0
    while len(hashed_transactions) > 1:
        if len(hashed_transactions) % 2 != 0:
            hashed_transactions.append(hashed_transactions[-1])

        new_transactions = []
        for i in range(0, len(hashed_transactions), 2):
            combined = hashed_transactions[i] + hashed_transactions[i+1]
            hash_combined = hash(combined)
            new_transactions.append(hash_combined)
        hashed_transactions = new_transactions
        level += 1

    # return the root of the merkle tree
    return hashed_transactions[0]

In [ ]:
# Creating a Blockchain
blockchain = Blockchain()

In [ ]:
# Helper function to hash
def hash(value):
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

In [ ]:
# Mining a new block
def mine_block():
    if len(blockchain.transactions) > 0:
        previous_block = blockchain.get_previous_block()
        previous_hash = blockchain.hash(previous_block)
        proof = blockchain.proof_of_work(blockchain.get_temp_block(previous_hash))
        block = blockchain.create_block(proof)
        response = {'message': 'Congratulations, you just mined a block!',
                    'index': block['index'],
                    'timestamp': block['timestamp'],
                    'nonce': block['nonce'],
                    'previous_hash': block['previous_hash']}
    else:
        response = {'message': 'No transactions to mine'}
    return response

In [ ]:
# Getting the full Blockchain
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return response

In [ ]:
# Checking if the Blockchain is valid
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'All good. The Blockchain is valid.'}
    else:
        response = {'message': 'Houston, we have a problem. The Blockchain is not valid.'}
    return response

In [ ]:
# Method to create and append transaction to blockchain
def create_transaction():
    sender = input("Enter sender: ")
    receiver = input("Enter receiver: ")
    amount = int(input("Enter amount: "))
    return blockchain.create_transaction(sender, receiver, amount)

In [ ]:
print("Functions Menu:")
print("===============")
print("1. Mine a block")
print("2. Create a transaction")
print("3. Display the chain")
print("4. Check the validity of the chain")
choice = int(input("Enter your choice: "))

if choice == 1:
    print(mine_block())
elif choice == 2:
    print(create_transaction())
elif choice == 3:
    print(get_chain())
elif choice == 4:
    print(is_valid())
else:
    print("Invalid choice")

Functions Menu:
1. Mine a block
2. Create a transaction
3. Display the chain
4. Check the validity of the chain
Enter your choice: 4
{'message': 'All good. The Blockchain is valid.'}
